In [ ]:
!pip3 install optuna==0.14.0
!pip3 install optkeras==0.0.7
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 91 kB 4.1 MB/s 
     |████████████████████████████████| 81 kB 7.9 MB/s 
     |████████████████████████████████| 209 kB 47.0 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 78 kB 7.5 MB/s 
     |████████████████████████████████| 112 kB 86.8 MB/s 
     |████████████████████████████████| 147 kB 63.5 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
  Created wheel for optuna: filename=optuna-0.14.0-py3-none-any.whl size=125709 sha256=83d3b5ff91bdbf1eb9ef5a72580f38d80574890171d565ec402fb39445c77219
  Stored in directory: /root/.cache/pip/wheels/c1/41/64/03b183676c5d5e978de160cab6268d5b4fb095dff63f720e01
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=bd2cf62c5ca3b6a42900b29264bff4c258e657024607754ee0aab0a12b362331
  Stored in directory: /root/.ca

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 78.7 MB/s 


# New part

In [ ]:
import numpy as np
import pandas as pd
import math
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Conv2D, Conv1D
from keras.layers import MaxPooling1D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import keras.backend as K

In [ ]:
import keras
print('Keras', keras.__version__)

import tensorflow as tf
print('TensorFlow', tf.__version__)

# import Optuna and OptKeras after Keras
import optuna 
print('Optuna', optuna.__version__)

from optkeras.optkeras import OptKeras
import optkeras
print('OptKeras', optkeras.__version__)

Keras 2.8.0
TensorFlow 2.8.2
Optuna 0.14.0
OptKeras 0.0.7


In [ ]:
#train_df = pd.read_parquet("/content/drive/MyDrive/New Data/train_main.parquet.snappy",engine='fastparquet')
#test_df = pd.read_parquet("/content/drive/MyDrive/New Data/val_main.parquet.snappy",engine='fastparquet')
#val_df = pd.read_parquet("/content/drive/MyDrive/New Data/test_main.parquet.snappy",engine='fastparquet')

In [ ]:
train_df.shape

NameError: ignored

##New Part for data

In [ ]:
train_df = pd.read_parquet("/content/drive/MyDrive/New Data/sub_data/train_main.parquet.snappy",engine='fastparquet')
test_df = pd.read_parquet("/content/drive/MyDrive/New Data/sub_data/test_main.parquet.snappy",engine='fastparquet')

In [19]:
test_df

,r_id,r_stars,r_stars_square,r_length,u_friends_count,u_review_count,u_month_age,b_stars,b_review_count,r_sen,r_sub,r_rea,r_useful
0,4195320,4,16,86,163,40,27.298941,4.0,445,0.117031,0.361875,86.10,1
1,4406379,5,25,28,18,115,0.001011,4.0,636,-0.140000,0.133333,96.48,3
2,1172072,5,25,76,69,1,0.000002,2.5,15,0.256723,0.355398,69.07,1
3,1949778,1,1,186,23,6,23.288288,3.5,185,-0.209252,0.378994,82.54,1
4,4377517,5,25,57,1,25,48.728723,4.5,227,0.295040,0.454762,91.11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5373252,5,25,25,288,2,24.026242,3.5,117,0.775000,0.662500,64.04,1
99996,4500278,5,25,76,21,59,44.444338,4.0,532,0.325664,0.561008,71.48,1
99997,4981139,5,25,65,236,143,4.868817,4.0,7568,0.097821,0.502692,83.66,2
99998,3032513,1,1,33,2,28,0.002720,3.0,80,0.300000,0.306250,100.04,3


In [ ]:
# prepare the cross-validation procedure
kfold = KFold(n_splits=3, shuffle=True)

In [ ]:
training=train_df.drop(labels="r_id", axis=1)
testing=test_df.drop(labels="r_id", axis=1)
#validing=val_df.drop(labels="r_id", axis=1)

training = training.iloc[: , 0:11]
testing = testing.iloc[: , 0:11]
#validing = validing.iloc[: , 0:11]

#labelsForTrain=train_df.iloc[: , -1]
#labelsForValid=test_df.iloc[: , -1]
#labelsForTest=val_df.iloc[: , -1]
#input_shape = training.shape

In [ ]:
#scaler = StandardScaler()

#std_train_df2=scaler.fit_transform(training)
#std_test_df2=scaler.transform(testing)
#std_val_df2=scaler.transform(validing)

In [ ]:
scaler = StandardScaler()

# scale skewed and target features
std_train_df = train_df.copy(deep=True)
std_train_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']] = scaler.fit_transform(train_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']])
std_test_df = test_df.copy(deep=True)
std_test_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']] = scaler.transform(test_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']])
#std_val_df = val_df.copy(deep=True)
#std_val_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']] = scaler.transform(val_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']])

std_train_df=std_train_df.drop(labels="r_id", axis=1)
std_test_df=std_test_df.drop(labels="r_id", axis=1)
#std_val_df=std_val_df.drop(labels="r_id", axis=1)

#std_train_df=scaler.fit_transform(std_train_df)
#std_test_df=scaler.transform(std_test_df)
#std_val_df=scaler.transform(std_val_df)

std_train_df

,r_stars,r_stars_square,r_length,u_friends_count,u_review_count,u_month_age,b_stars,b_review_count,r_sen,r_sub,r_rea,r_useful
0,-1.694060,-1.494530,-0.297754,-0.076615,0.335185,-1.101172,-0.308494,-0.472926,-0.015873,0.529586,0.546201,3
1,0.904508,0.998977,1.233089,-0.329431,-0.373350,-0.300852,0.322022,0.958189,0.320463,0.549522,0.969516,1
2,-1.694060,-1.494530,0.798190,0.176200,-0.379321,-0.069431,-0.308494,-0.052545,-0.090963,0.533279,-0.170954,1
3,0.904508,0.998977,0.572043,-0.012104,-0.245973,-1.223231,0.322022,0.173580,0.217308,0.556899,0.192674,1
4,0.904508,0.998977,-0.854423,-0.329431,-0.377331,-0.107964,-2.200042,-0.497208,0.408333,0.608333,-0.053417,2
...,...,...,...,...,...,...,...,...,...,...,...,...
399995,-1.044418,-1.182842,7.713075,-0.285842,-0.315632,0.268719,-0.939010,-0.465338,-0.025891,0.270243,1.025529,10
399996,-1.694060,-1.494530,-0.341243,-0.179485,-0.377331,-0.135109,0.952538,-0.459267,0.046314,0.397489,0.099013,1
399997,-1.044418,-1.182842,-0.428223,-0.315482,-0.216119,-0.683035,-1.569526,-0.460785,0.231790,0.673650,-0.509789,1
399998,-1.694060,-1.494530,0.084957,-0.329431,-0.379321,-0.425276,-1.569526,-0.298400,0.029268,0.306944,1.904296,1


In [ ]:
std_train_df.shape

(400000, 12)

In [ ]:
training = std_train_df.iloc[: , 0:11]
testing = std_test_df.iloc[: , 0:11]
#validing = std_val_df.iloc[: , 0:11]
labelsForTrain=std_train_df.iloc[: , -1]
#labelsForValid=std_val_df.iloc[: , -1]
labelsForTest=std_test_df.iloc[: , -1]
input_shape = training.shape

In [ ]:
training.shape

(400000, 11)

# Another test

In [ ]:
from keras.layers import Input
from keras.layers.core import *
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop, SGD
from keras.models import Model
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import keras.backend as K
import optuna
import math

In [ ]:
def create_model(activation, num_hidden_layer, num_hidden_unit):
  inputs = Input(shape=(training.shape[1],))
  model = inputs
  for i in range(1,num_hidden_layer):
    model = Dense(num_hidden_unit, activation=activation,)(model)
        
        
  model = Dense(1,)(model)
  model = Model(inputs, model)

  return model
    

In [ ]:
#def create_model_cnn(activation, num_hidden_layer, num_hidden_unit, kernel_size, filter):
#  inputs = Input(shape=(training.shape[1],))
#  model = inputs
#  model = Sequential()
#  for j in range(1):
#    model = Conv1D(filter, kernel_size, activation=activation, input_shape=(11,1))
#    model = Flatten()
#    for i in range(1,num_hidden_layer):
#      model = Dense(num_hidden_unit, activation=activation,)(model)
#  model = Dense(1, activation='relu')(model)
#  model = Model(inputs, model)

#  return model

In [ ]:
def create_model_cnn(activation, num_hidden_layer, num_hidden_unit, kernel_size, filter):
  model_cnn = Sequential()
  model_cnn.add(Conv1D(filter, kernel_size, activation=activation, input_shape=(11,1)))
  model_cnn.add(MaxPooling1D())
  model_cnn.add(Flatten())
  for i in range(1,num_hidden_layer):
    model_cnn.add(Dense(num_hidden_unit, activation=activation,))

  model_cnn.add(Dense(1))

  return model_cnn

In [ ]:
def objective_cnn(trial):
  K.clear_session()
    
  activation = trial.suggest_categorical('activation',['relu','tanh','linear'])
  optimizer = trial.suggest_categorical('optimizer',['adam','rmsprop','adagrad', 'sgd'])
    
  num_hidden_layer = trial.suggest_int('num_hidden_layer',1,5)
  num_hidden_unit = trial.suggest_int('num_hidden_unit',10,100)
  kernel_size = trial.suggest_int('kernel_size',1,5)
  filter = trial.suggest_int('filter',1,100)
    
  learning_rate = trial.suggest_loguniform('learning_rate', 0.00001,0.1)
  if optimizer == 'adam':
    optimizer = Adam(learning_rate=learning_rate)
  elif optimizer == 'adagrad':
    optimizer = Adagrad(learning_rate=learning_rate)
  elif optimizer =='rmsprop':
    optimizer = RMSprop(learning_rate=learning_rate)
  elif optimizer =='sgd':
    optimizer = SGD(learning_rate=learning_rate)
    
  num_folds = 3
  kfold=KFold(n_splits=3,shuffle=True)
  fold_no=1
  loss_per_fold = []
  es = EarlyStopping(monitor='mse', patience=50)
  model = create_model_cnn(activation, num_hidden_layer, num_hidden_unit, kernel_size, filter)
  model_list_cnn.append(model)
  model.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
  for train,test in kfold.split(training,labelsForTrain):
    scores=model.evaluate(testing,labelsForTest,verbose=0)

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
  
    # Fit data to model
    history = model.fit(training, labelsForTrain,
                batch_size=20,
                epochs=10,
                verbose=2,
                callbacks=[es])
    
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}')
    loss_per_fold.append(scores[0])
    # Increase fold number
    fold_no = fold_no + 1  
  
  history_list_cnn.append(history)
    
  mse = np.array(history.history['mse'])
    
  return mse[-1]

In [ ]:
def objective(trial):
  K.clear_session()
    
  activation = trial.suggest_categorical('activation',['relu','tanh','linear'])
  optimizer = trial.suggest_categorical('optimizer',['adam','rmsprop','adagrad', 'sgd'])
    
  num_hidden_layer = trial.suggest_int('num_hidden_layer',1,3)
  num_hidden_unit = trial.suggest_int('num_hidden_unit',10,500)
    
  learning_rate = trial.suggest_loguniform('learning_rate', 0.00001,0.1)
  if optimizer == 'adam':
    optimizer = Adam(learning_rate=learning_rate)
  elif optimizer == 'adagrad':
    optimizer = Adagrad(learning_rate=learning_rate)
  elif optimizer =='rmsprop':
    optimizer = RMSprop(learning_rate=learning_rate)
  elif optimizer =='sgd':
    optimizer = SGD(learning_rate=learning_rate)
    
  num_folds = 3
  kfold=KFold(n_splits=3,shuffle=True)
  fold_no=1
  loss_per_fold = []
  es = EarlyStopping(monitor='mse', patience=50)
  model = create_model(activation, num_hidden_layer, num_hidden_unit)
  model_list.append(model)
  model.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
  for train,test in kfold.split(training,labelsForTrain):
    scores=model.evaluate(testing,labelsForTest,verbose=0)

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
  
    # Fit data to model
    history = model.fit(training, labelsForTrain,
                batch_size=20,
                epochs=10,
                verbose=2,
                callbacks=[es])
    
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}')
    loss_per_fold.append(scores[0])
    # Increase fold number
    fold_no = fold_no + 1  
  
  history_list.append(history)
    
  mse = np.array(history.history['mse'])
    
  return mse[-1]

In [ ]:
model_list_cnn=[]
history_list_cnn=[]
study_name = 'CNN_study'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
study.optimize(objective_cnn, n_trials=50, )

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 31s - loss: 1005.5142 - mse: 1005.5142 - mae: 7.8811 - 31s/epoch - 2ms/step
Epoch 2/10
20000/20000 - 30s - loss: 1597.7765 - mse: 1597.7765 - mae: 9.2947 - 30s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 30s - loss: 2279.2959 - mse: 2279.2959 - mae: 10.1906 - 30s/epoch - 2ms/step
Epoch 4/10
20000/20000 - 32s - loss: 1745.0280 - mse: 1745.0280 - mae: 9.8582 - 32s/epoch - 2ms/step
Epoch 5/10
20000/20000 - 31s - loss: 1636.7992 - mse: 1636.7992 - mae: 9.5566 - 31s/epoch - 2ms/step
Epoch 6/10
20000/20000 - 30s - loss: 2039.6914 - mse: 2039.6914 - mae: 10.0259 - 30s/epoch - 2ms/step
Epoch 7/10
20000/20000 - 29s - loss: 2213.4902 - mse: 2213.4902 - mae: 9.6500 - 29s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 28s - loss: 1950.7869 - mse: 1950.7869 - mae: 10.0176 - 28s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 28s - loss: 2025.6382 - mse: 2025.6382 - mae: 9.9798 - 28s/epoch - 

[I 2022-09-28 21:22:10,524] Finished trial#0 resulted in value: 2390.018310546875. Current best value is 2390.018310546875 with parameters: {'activation': 'linear', 'optimizer': 'rmsprop', 'num_hidden_layer': 4, 'num_hidden_unit': 79, 'kernel_size': 5, 'filter': 99, 'learning_rate': 0.025805989874184886}.


Score for fold 3: loss of 36.11547088623047
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 23s - loss: 14.6577 - mse: 14.6577 - mae: 1.6834 - 23s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 22s - loss: 14.6124 - mse: 14.6124 - mae: 1.6809 - 22s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 22s - loss: 14.6114 - mse: 14.6114 - mae: 1.6818 - 22s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 21s - loss: 14.6150 - mse: 14.6150 - mae: 1.6818 - 21s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 22s - loss: 14.6059 - mse: 14.6059 - mae: 1.6823 - 22s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 22s - loss: 14.6023 - mse: 14.6023 - mae: 1.6829 - 22s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 22s - loss: 14.6001 - mse: 14.6001 - mae: 1.6830 - 22s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 22s - loss: 14.6009 - mse: 14.6009 - mae: 1.6826 - 22s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 22s - loss: 14.6004 - mse: 14.6004 - mae: 1.6832 - 22s/epo

[I 2022-09-28 21:32:58,263] Finished trial#1 resulted in value: 14.593823432922363. Current best value is 14.593823432922363 with parameters: {'activation': 'linear', 'optimizer': 'sgd', 'num_hidden_layer': 3, 'num_hidden_unit': 64, 'kernel_size': 1, 'filter': 31, 'learning_rate': 0.0019453864051127093}.


Score for fold 3: loss of 19.58966827392578
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 28s - loss: 14.4640 - mse: 14.4640 - mae: 1.6519 - 28s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 28s - loss: 14.3155 - mse: 14.3155 - mae: 1.6486 - 28s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 28s - loss: 14.3014 - mse: 14.3015 - mae: 1.6482 - 28s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 28s - loss: 14.2870 - mse: 14.2870 - mae: 1.6476 - 28s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 28s - loss: 14.2828 - mse: 14.2828 - mae: 1.6472 - 28s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 29s - loss: 14.2842 - mse: 14.2842 - mae: 1.6468 - 29s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 28s - loss: 14.2805 - mse: 14.2805 - mae: 1.6464 - 28s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 28s - loss: 14.2769 - mse: 14.2769 - mae: 1.6463 - 28s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 28s - loss: 14.2762 - mse: 14.2762 - mae: 1.6462 - 28s/epo

...

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 30s - loss: 14.4315 - mse: 14.4315 - mae: 1.6642 - 30s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 28s - loss: 14.2453 - mse: 14.2453 - mae: 1.6489 - 28s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 30s - loss: 14.1972 - mse: 14.1972 - mae: 1.6464 - 30s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 30s - loss: 14.1644 - mse: 14.1644 - mae: 1.6428 - 30s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 30s - loss: 14.1318 - mse: 14.1318 - mae: 1.6410 - 30s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 30s - loss: 14.1041 - mse: 14.1041 - mae: 1.6404 - 30s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 29s - loss: 14.0539 - mse: 14.0539 - mae: 1.6387 - 29s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 28s - loss: 14.0314 - mse: 14.0314 - mae: 1.6389 - 28s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 30s - loss: 14.0217 - mse: 14.0217 - mae: 1.6376 - 30s/epoch - 2ms/step
Epoch 10/10
20000/20000 - 30s 

[I 2022-09-29 07:36:29,470] Finished trial#36 resulted in value: 13.89054012298584. Current best value is 12.688554763793945 with parameters: {'activation': 'tanh', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 85, 'kernel_size': 4, 'filter': 77, 'learning_rate': 0.00041922751339262246}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 48s - loss: 14.1793 - mse: 14.1793 - mae: 1.6272 - 48s/epoch - 2ms/step
Epoch 2/10
20000/20000 - 48s - loss: 13.8856 - mse: 13.8856 - mae: 1.6138 - 48s/epoch - 2ms/step
Epoch 3/10
20000/20000 - 48s - loss: 13.7580 - mse: 13.7580 - mae: 1.6069 - 48s/epoch - 2ms/step
Epoch 4/10
20000/20000 - 48s - loss: 13.6403 - mse: 13.6403 - mae: 1.6031 - 48s/epoch - 2ms/step
Epoch 5/10
20000/20000 - 48s - loss: 13.5624 - mse: 13.5624 - mae: 1.5992 - 48s/epoch - 2ms/step
Epoch 6/10
20000/20000 - 49s - loss: 13.5064 - mse: 13.5064 - mae: 1.5969 - 49s/epoch - 2ms/step
Epoch 7/10
20000/20000 - 51s - loss: 13.4579 - mse: 13.4579 - mae: 1.5949 - 51s/epoch - 3ms/step
Epoch 8/10
20000/20000 - 49s - loss: 13.4217 - mse: 13.4217 - mae: 1.5943 - 49s/epoch - 2ms/step
Epoch 9/10
20000/20000 - 49s - loss: 13.3752 - mse: 13.3752 - mae: 1.5931 - 49s/epoch - 2ms/step
Epoch 10/10
20000/20000 - 48s 

[I 2022-09-29 08:00:43,942] Finished trial#37 resulted in value: 12.9368896484375. Current best value is 12.688554763793945 with parameters: {'activation': 'tanh', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 85, 'kernel_size': 4, 'filter': 77, 'learning_rate': 0.00041922751339262246}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 45s - loss: 14.4143 - mse: 14.4143 - mae: 1.6382 - 45s/epoch - 2ms/step
Epoch 2/10
20000/20000 - 47s - loss: 13.9863 - mse: 13.9863 - mae: 1.6172 - 47s/epoch - 2ms/step
Epoch 3/10
20000/20000 - 48s - loss: 13.8980 - mse: 13.8980 - mae: 1.6123 - 48s/epoch - 2ms/step
Epoch 4/10
20000/20000 - 47s - loss: 13.8535 - mse: 13.8535 - mae: 1.6099 - 47s/epoch - 2ms/step
Epoch 5/10
20000/20000 - 48s - loss: 13.8152 - mse: 13.8152 - mae: 1.6079 - 48s/epoch - 2ms/step
Epoch 6/10
20000/20000 - 48s - loss: 13.7887 - mse: 13.7887 - mae: 1.6059 - 48s/epoch - 2ms/step
Epoch 7/10
20000/20000 - 52s - loss: 13.7702 - mse: 13.7702 - mae: 1.6049 - 52s/epoch - 3ms/step
Epoch 8/10
20000/20000 - 52s - loss: 13.7391 - mse: 13.7391 - mae: 1.6038 - 52s/epoch - 3ms/step
Epoch 9/10
20000/20000 - 48s - loss: 13.7193 - mse: 13.7193 - mae: 1.6027 - 48s/epoch - 2ms/step
Epoch 10/10
20000/20000 - 48s 

## Select best CNN model

In [ ]:
#Without maxpooling:{'activation': 'tanh', 'optimizer': 'adam', 'num_hidden_layer': 4, 'num_hidden_unit': 79, 'kernel_size': 3, 'filter': 1, 'learning_rate': 0.0006459535879377347}
#{'activation': 'tanh', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 85, 'kernel_size': 4, 'filter': 77, 'learning_rate': 0.00041922751339262246}
model_cnn_best = create_model_cnn(activation='tanh', num_hidden_layer=5, num_hidden_unit=85, kernel_size=4, filter=77)
model_cnn_best.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 8, 77)             385       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4, 77)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 308)               0         
                                                                 
 dense (Dense)               (None, 85)                26265     
                                                                 
 dense_1 (Dense)             (None, 85)                7310      
                                                                 
 dense_2 (Dense)             (None, 85)                7310      
                                                        

In [ ]:
num_folds = 3
kfold=KFold(n_splits=3,shuffle=True)
fold_no=1
loss_per_fold = []
model_cnn_best = create_model_cnn(activation='tanh', num_hidden_layer=5, num_hidden_unit=85, kernel_size=4, filter=77)
model_cnn_best.summary()
model_cnn_best.compile(loss='mse',optimizer='adam',metrics=['mse','mae'])
for train,test in kfold.split(training,labelsForTrain):
  scores=model_cnn_best.evaluate(testing,labelsForTest,verbose=0)

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  
  # Fit data to model
  history = model_cnn_best.fit(training, labelsForTrain,
              batch_size=20,
              #The result from epoch 20 and epoch 200 don't make too much differenct
              epochs=100,
              verbose=2,)
    
  print(f'Score for fold {fold_no}: {model_cnn_best.metrics_names[0]} of {scores[0]}')
  loss_per_fold.append(scores[0])
  # Increase fold number
  fold_no = fold_no + 1 

model_cnn_best.compile(loss='mse',optimizer='adam',metrics=['mse','mae'])
history = model_cnn_best.fit(training, labelsForTrain,
                batch_size=20,
                epochs=200,
                verbose=2)
    

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 8, 77)             385       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4, 77)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 308)               0         
                                                                 
 dense (Dense)               (None, 85)                26265     
                                                                 
 dense_1 (Dense)             (None, 85)                7310      
                                                                 
 dense_2 (Dense)             (None, 85)                7310      
                                                        

In [18]:
results = model_cnn_best.evaluate(testing, labelsForTest, batch_size=20)

5000/5000 [==============================] - 6s 1ms/step - loss: 20.3916 - mse: 20.3916 - mae: 1.8200


In [ ]:
summation = 0  #variable to store the summation of differences
test_predictions_cnn = model_cnn_best.predict(testing).flatten()
n = len(test_predictions_cnn) #finding total number of items in list
for i in range (0,n):  #looping through each element of the list
  difference = labelsForTest[i] - test_predictions_cnn[i]  #finding the difference between observed and predicted value
  squared_difference = difference**2  #taking square of the differene 
  summation = summation + squared_difference  #taking a sum of all the differences
MSE = summation/n  #dividing summation by total values to obtain average
RMSE = math.sqrt(MSE)
print ("The Mean Square Error for test (CNN) is: " , MSE)
print ("The Root Mean Square Error for test (CNN) is: " , RMSE)

The Mean Square Error for test (CNN) is:  18.479924529298682
The Root Mean Square Error for test (CNN) is:  4.298828273994983


In [ ]:
model_list=[]
history_list=[]
study_name = 'NN_study'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
#15.1
study.optimize(objective, n_trials=50, )

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 26s - loss: 14.7860 - mse: 14.7860 - mape: 81.2645 - 26s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 26s - loss: 14.6950 - mse: 14.6950 - mape: 80.9104 - 26s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 25s - loss: 14.6744 - mse: 14.6744 - mape: 80.6904 - 25s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 25s - loss: 14.6636 - mse: 14.6636 - mape: 80.7697 - 25s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 26s - loss: 14.6661 - mse: 14.6661 - mape: 80.6036 - 26s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 25s - loss: 14.6694 - mse: 14.6694 - mape: 80.7526 - 25s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 25s - loss: 14.6539 - mse: 14.6539 - mape: 80.6988 - 25s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 25s - loss: 14.6513 - mse: 14.6513 - mape: 80.6673 - 25s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 26s - loss: 14.6469 - mse: 14.6469 - mape: 80.8592 - 26s/epoch - 1ms/step
Epoch 10/10


[I 2022-09-27 11:06:34,306] Finished trial#0 resulted in value: 14.641683578491211. Current best value is 14.641683578491211 with parameters: {'activation': 'linear', 'optimizer': 'rmsprop', 'num_hidden_layer': 4, 'num_hidden_unit': 84, 'learning_rate': 0.003470769400535044}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 25s - loss: 14.8915 - mse: 14.8915 - mape: 84.9923 - 25s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 24s - loss: 14.7340 - mse: 14.7340 - mape: 85.6471 - 24s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 24s - loss: 14.7298 - mse: 14.7298 - mape: 85.5716 - 24s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 24s - loss: 14.7173 - mse: 14.7173 - mape: 85.4801 - 24s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 24s - loss: 14.7254 - mse: 14.7254 - mape: 85.7689 - 24s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 24s - loss: 14.7013 - mse: 14.7013 - mape: 85.7187 - 24s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 24s - loss: 14.7400 - mse: 14.7400 - mape: 86.0393 - 24s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 24s - loss: 14.7284 - mse: 14.7284 - mape: 85.8007 - 24s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 24s - loss: 14.7324 - mse: 14.7324 - mape: 85.9275 - 24s/epoch - 1ms/step
Epoch 10/10


[I 2022-09-27 11:18:51,246] Finished trial#1 resulted in value: 14.742877960205078. Current best value is 14.641683578491211 with parameters: {'activation': 'linear', 'optimizer': 'rmsprop', 'num_hidden_layer': 4, 'num_hidden_unit': 84, 'learning_rate': 0.003470769400535044}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 21s - loss: 17.0342 - mse: 17.0342 - mape: 73.8339 - 21s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 21s - loss: 14.7361 - mse: 14.7361 - mape: 80.4231 - 21s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 21s - loss: 14.6013 - mse: 14.6013 - mape: 80.4133 - 21s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 21s - loss: 14.5268 - mse: 14.5268 - mape: 80.3706 - 21s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 21s - loss: 14.4750 - mse: 14.4750 - mape: 80.3662 - 21s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 21s - loss: 14.4326 - mse: 14.4326 - mape: 80.2567 - 21s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 21s - loss: 14.3981 - mse: 14.3981 - mape: 80.0197 - 21s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 20s - loss: 14.3649 - mse: 14.3649 - mape: 79.9947 - 20s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 21s - loss: 14.3330 - mse: 14.3330 - mape: 80.0816 - 21s/epoch - 1ms/step
Epoch 10/10


...

[I 2022-09-27 21:47:50,527] Finished trial#47 resulted in value: 12.857813835144043. Current best value is 12.554261207580566 with parameters: {'activation': 'tanh', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 68, 'learning_rate': 0.00036577231287438313}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 32s - loss: 15.2360 - mse: 15.2360 - mape: 80.0052 - 32s/epoch - 2ms/step
Epoch 2/10
20000/20000 - 32s - loss: 14.3695 - mse: 14.3695 - mape: 80.1033 - 32s/epoch - 2ms/step
Epoch 3/10
20000/20000 - 32s - loss: 14.2167 - mse: 14.2167 - mape: 79.6103 - 32s/epoch - 2ms/step
Epoch 4/10
20000/20000 - 32s - loss: 14.1183 - mse: 14.1183 - mape: 79.1894 - 32s/epoch - 2ms/step
Epoch 5/10
20000/20000 - 32s - loss: 14.0533 - mse: 14.0533 - mape: 78.9209 - 32s/epoch - 2ms/step
Epoch 6/10
20000/20000 - 32s - loss: 14.0039 - mse: 14.0039 - mape: 78.6484 - 32s/epoch - 2ms/step
Epoch 7/10
20000/20000 - 32s - loss: 13.9697 - mse: 13.9697 - mape: 78.4300 - 32s/epoch - 2ms/step
Epoch 8/10
20000/20000 - 32s - loss: 13.9395 - mse: 13.9395 - mape: 78.2842 - 32s/epoch - 2ms/step
Epoch 9/10
20000/20000 - 32s - loss: 13.9148 - mse: 13.9148 - mape: 78.1720 - 32s/epoch - 2ms/step
Epoch 10/10


[I 2022-09-27 22:04:35,016] Finished trial#48 resulted in value: 13.716154098510742. Current best value is 12.554261207580566 with parameters: {'activation': 'tanh', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 68, 'learning_rate': 0.00036577231287438313}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 33s - loss: 14.6964 - mse: 14.6964 - mape: 80.1054 - 33s/epoch - 2ms/step
Epoch 2/10
20000/20000 - 33s - loss: 14.1313 - mse: 14.1313 - mape: 79.0275 - 33s/epoch - 2ms/step
Epoch 3/10
20000/20000 - 34s - loss: 14.0052 - mse: 14.0052 - mape: 78.5805 - 34s/epoch - 2ms/step
Epoch 4/10
20000/20000 - 34s - loss: 13.9346 - mse: 13.9346 - mape: 78.2646 - 34s/epoch - 2ms/step
Epoch 5/10
20000/20000 - 33s - loss: 13.8907 - mse: 13.8907 - mape: 78.1063 - 33s/epoch - 2ms/step
Epoch 6/10
20000/20000 - 33s - loss: 13.8602 - mse: 13.8602 - mape: 77.9916 - 33s/epoch - 2ms/step
Epoch 7/10
20000/20000 - 33s - loss: 13.8381 - mse: 13.8381 - mape: 77.8805 - 33s/epoch - 2ms/step
Epoch 8/10
20000/20000 - 32s - loss: 13.8107 - mse: 13.8107 - mape: 77.8492 - 32s/epoch - 2ms/step
Epoch 9/10
20000/20000 - 32s - loss: 13.7895 - mse: 13.7895 - mape: 77.7222 - 32s/epoch - 2ms/step
Epoch 10/10


[I 2022-09-27 22:20:50,043] Finished trial#49 resulted in value: 13.46130657196045. Current best value is 12.554261207580566 with parameters: {'activation': 'tanh', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 68, 'learning_rate': 0.00036577231287438313}.


In [ ]:
num_folds = 3
kfold=KFold(n_splits=3,shuffle=True)
#{'activation': 'tanh', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 68, 'learning_rate': 0.00036577231287438313}
fold_no=1
loss_per_fold = []
es = EarlyStopping(monitor='mse', patience=50)
model_nn_best = create_model(activation='tanh', num_hidden_layer=5, num_hidden_unit=68)
model_nn_best.summary()
model_nn_best.compile(loss='mse',optimizer='adam',metrics=['mse','mae'])
for train,test in kfold.split(training,labelsForTrain):
  scores=model_nn_best.evaluate(testing,labelsForTest,verbose=0)

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  
  # Fit data to model
  history = model_nn_best.fit(training, labelsForTrain,
              batch_size=20,
              #The result from epoch 20 and epoch 200 don't make too much differenct
              epochs=20,
              verbose=2,)
    
  print(f'Score for fold {fold_no}: {model_nn_best.metrics_names[0]} of {scores[0]}')
  loss_per_fold.append(scores[0])
  # Increase fold number
  fold_no = fold_no + 1  


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 11)]              0         
                                                                 
 dense (Dense)               (None, 68)                816       
                                                                 
 dense_1 (Dense)             (None, 68)                4692      
                                                                 
 dense_2 (Dense)             (None, 68)                4692      
                                                                 
 dense_3 (Dense)             (None, 68)                4692      
                                                                 
 dense_4 (Dense)             (None, 1)                 69        
                                                                 
Total params: 14,961
Trainable params: 14,961
Non-trainable p

In [ ]:
results = model_nn_best.evaluate(testing, labelsForTest, batch_size=10)

10000/10000 [==============================] - 20s 2ms/step - loss: 18.3104 - mse: 18.3104 - mae: 1.6508


In [ ]:
summation = 0  #variable to store the summation of differences
test_predictions = model_nn_best.predict(testing).flatten()
n = len(test_predictions) #finding total number of items in list
for i in range (0,n):  #looping through each element of the list
  difference = labelsForTest[i] - test_predictions[i]  #finding the difference between observed and predicted value
  squared_difference = difference**2  #taking square of the differene 
  summation = summation + squared_difference  #taking a sum of all the differences
MSE = summation/n  #dividing summation by total values to obtain average
RMSE = math.sqrt(MSE)
print ("The Mean Square Error for test is: " , MSE)
print ("The Root Mean Square Error for test is: " , RMSE)

The Mean Square Error for test is:  18.344845788207742
The Root Mean Square Error for test is:  4.2830883470000645


In [ ]:
num_folds = 3
kfold=KFold(n_splits=3,shuffle=True)
fold_no=1
loss_per_fold = []
for train,test in kfold.split(training,labelsForTrain):
  scores=model_list[43].evaluate(testing,labelsForTest,verbose=0)
  model.compile(loss='mse',optimizer='adam',metrics=['mse'])

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
 
  # Fit data to model
  history = model.fit(training, labelsForTrain,
              batch_size=20,
              epochs=10,
              verbose=2)
  
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}')
  loss_per_fold.append(scores[0])
  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 24s - loss: 14.0593 - mse: 14.0593 - 24s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 24s - loss: 13.8439 - mse: 13.8439 - 24s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 24s - loss: 13.7533 - mse: 13.7533 - 24s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 24s - loss: 13.6438 - mse: 13.6438 - 24s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 24s - loss: 13.6091 - mse: 13.6091 - 24s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 24s - loss: 13.5218 - mse: 13.5218 - 24s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 24s - loss: 13.4684 - mse: 13.4684 - 24s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 24s - loss: 13.4305 - mse: 13.4305 - 24s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 23s - loss: 13.3944 - mse: 13.3944 - 23s/epoch - 1ms/step
Epoch 10/10
20000/20000 - 24s - loss: 13.3802 - mse: 13.3802 - 24s/epoch - 1ms/step
Score for fold 1: loss of 18.295801162719727
---------------------------

In [ ]:
print(study.best_params)
print('')
print(study.best_value)

{'activation': 'relu', 'optimizer': 'adam', 'num_hidden_layer': 4, 'num_hidden_unit': 59, 'learning_rate': 0.0024291088439318004}

13.41602611541748


## Read New Data

In [70]:
train_df = pd.read_parquet("/content/drive/MyDrive/New Data/18features/18_train_main.parquet.snappy",engine='fastparquet')
test_df = pd.read_parquet("/content/drive/MyDrive/New Data/18features/18_test_main.parquet.snappy",engine='fastparquet')
# prepare the cross-validation procedure
kfold = KFold(n_splits=3, shuffle=True)

training_withaim=train_df.drop(labels="r_id", axis=1)
testing_withaim=test_df.drop(labels="r_id", axis=1)

#Check the NaN in data and drop them
imp_train=SimpleImputer(missing_values=np.NaN)
training=pd.DataFrame(imp_train.fit_transform(training_withaim))

imp_test=SimpleImputer(missing_values=np.NaN)
testing=pd.DataFrame(imp_test.fit_transform(testing_withaim))
print(training)
# There aren't any nan data in the dataframe 
training.isnull().values.sum()
testing.isnull().values.sum()
training = training.iloc[: , 0:18]
testing = testing.iloc[: , 0:18]
scaler = StandardScaler()

# scale skewed and target features
std_train_df = train_df.copy(deep=True)
std_train_df = scaler.fit_transform(training)
std_test_df = test_df.copy(deep=True)
std_test_df = scaler.transform(testing)
#std_test_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']] = scaler.transform(test_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']])

print(std_train_df[0])
std_train_df = pd.DataFrame(std_train_df)
std_test_df = pd.DataFrame(std_test_df)
training = std_train_df.iloc[: , 0:18]
testing = std_test_df.iloc[: , 0:18]
labelsForTrain=training_withaim.iloc[: , -1]
labelsForTest=testing_withaim.iloc[: , -1]

         0     1      2      3       4           5         6    7     8   \
0       5.0  25.0   80.0  173.0   129.0  108.203988  0.178295  3.0   6.0   
1       4.0  16.0  171.0  128.0   330.0   58.652416  0.909091  7.0  21.0   
2       3.0   9.0  114.0  277.0  1064.0   64.378573  1.753759  2.0  57.0   
3       5.0  25.0   34.0   35.0    59.0  102.077493  0.067797  0.0   0.0   
4       1.0   1.0  261.0  137.0     5.0   44.994772  0.000000  0.0   0.0   
...     ...   ...    ...    ...     ...         ...       ...  ...   ...   
399995  5.0  25.0  191.0    1.0    16.0    0.000110  0.000000  0.0   0.0   
399996  3.0   9.0  434.0  164.0   471.0   70.677502  0.116773  4.0  22.0   
399997  5.0  25.0  131.0  211.0     7.0   23.186602  0.142857  0.0   1.0   
399998  5.0  25.0   49.0    1.0    16.0    6.598250  0.062500  0.0   0.0   
399999  1.0   1.0   25.0    1.0    58.0   27.243015  0.034483  0.0   0.0   

          9       10   11      12   13     14        15        16     17    18  
0     

# Old Part (No need to use this ,contain the data with extrack RGB)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow as cv2
from itertools import product
from keras import models,layers
from PIL import Image
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv1D
import glob
import os
import csv
import math 
import json
import pandas as pd
import tensorflow as tf
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import optuna
from optkeras.optkeras import OptKeras

In [ ]:
#gpu_info = !nvidia-smi
#gpu_info = '\n'.join(gpu_info)
#if gpu_info.find('failed') >= 0:
#  print('Not connected to a GPU')
#else:
#  print(gpu_info)

Mon Aug 22 05:33:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#
#model = Sequential(
#    [
#     Dense(12, activation='relu', input_shape=(11,)),
#     Dense(12, activation='relu'),
#     Dense(training.size),
#     ])
#model.build(input_shape=(11,1))
#model.compile(optimizer='sgd', loss='mse',metrics=['accuracy'])
#model.summary()


#model = tf.keras.Sequential([
#    tf.keras.layers.Dense(12, activation='relu'),
#    tf.keras.layers.Dense(8, activation='relu'),
#    tf.keras.layers.Dense(1, activation='sigmoid')
#])
#model.build(input_shape=(13,2))
#model.compile(optimizer='sgd', loss='mse',metrics=['accuracy'])
#model.summary()



In [ ]:
#num_epochs = 50
#history = model.fit(training, labelsForTrain, epochs=num_epochs, validation_data=(validing, labelsForValid), verbose=2)

In [ ]:
#vocab_size = 50000
#embedding_dim = 16
#max_length = 200
#model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#    tf.keras.layers.Dense(24, activation='relu'),
#    tf.keras.layers.Dense(8, activation='relu'),
#    tf.keras.layers.Dense(1, activation='sigmoid')
#])


#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#model.summary()

In [ ]:
# Check list: 
#1. Review rating 
#2. Review Rating Extremity (?)
#3. Word Count
#4. User No of friends
#5. User review count
#6. User Age of account (months)
#7. Restaurant avg. rating
#8. Restaurant review count
#9. Review text total sentiment polarity score of each sentence / total # sentences (need to be done)
#10. Subjectivity score (subjective vs objective)
#11. Readability

Below is the code for extracting the RGB data

In [ ]:
#with open('output_file.csv', 'w', newline='') as f_output:
 #   csv_output = csv.writer(f_output)
  #  csv_output.writerow(["img_name", "R", "G", "B"])

with open("RGB.json", "w") as outfile:
    for img_path in glob.glob("/content/drive/MyDrive/photos/*.jpg"):
    #img_path = '/content/drive/MyDrive/test/--3P3fjPrScy06XVLi4vhg.jpg'
     try:
      image = Image.open(img_path)
      img_name = os.path.basename(img_path)
      # Number of prime color we gonna extrace

      num_colors=1

      #Generate small image for calculation
      small_image=image.resize((20,20))

      result=small_image.convert('P',palette=Image.ADAPTIVE,colors=num_colors)
      result=result.convert('RGB')

      main_colors = result.getcolors(80*80)
      main_rgb = main_colors[0][1]
      #csv_output.writerows([img_name, map(lambda x: [x], main_rgb)])
      # Data to be written
      dictionary ={
          "r_name": img_name,
          "R": main_rgb[0],
          "G": main_rgb[1],
          "B": main_rgb[2]
      }
      outfile.write('\n')
      json.dump(dictionary, outfile)
     except:
      print(img_name)

oIlZM9LN-Ol_WfQQQpg0Zw.jpg


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:1015: UserWarning: Couldn't allocate palette entry for transparency
  warnings.warn("Couldn't allocate palette entry for transparency")


0GJJhj1lh8V6zLeLkFwbyQ.jpg
Va9Zo6Y6zrWzpm9iV4GXZw.jpg
mKI1GpWQMs2rckjI3D9dhw.jpg
2q_8hrvKQCFD18MyJzoVmw.jpg
pm5uBNJCVRB3kZXq1eOyiA.jpg
6qqRxbjA2HquZZnNQrvxEQ.jpg
MDF6GetsTNRYWyIzzxea7A.jpg
HNgnhFdN15zIw8HzsRzWOA.jpg
Z7r1dYDC9eR6DQq01yGU-Q.jpg
O8eLVHCthPJuv02-EY7GdA.jpg
bWnSyaeOJgbykBL56ldM9w.jpg
ogQp0M7B1OiS8AqP55eirQ.jpg
A7FepQWBzC9eIXDUYImtbQ.jpg
HyFN7L1PYIeTTBTY2-G6ZQ.jpg
phGmoKO6rJpyQo8TDQofqQ.jpg
Qi-lxakzwDIPVRFPyvMpUQ.jpg
c9KWE6TeU4tRRkhxuYJCAQ.jpg
NX1a9ZNo8a34RuALiBqROw.jpg
lz-22DatK0pT7CkGLg7OgQ.jpg
EhqaIciplglkJBPvufErUA.jpg
JmWCtmZ1V3NrSTHs0mB4OA.jpg
-_seY1mjA62XKXwi5FZGBQ.jpg
Na1sCel7pdY2CRQnWBo84g.jpg
QuxU8mtIK7P-glLX7kZmog.jpg


In [ ]:
#data= pd.read_json('/content/RGB.json', lines=True, orient='records')
data= pd.read_json(open("/content/drive/MyDrive/RGB.json", "r"), lines = True)
data.head()

,photo_id,R,G,B
0,--0h6FMC0V8aMtKQylojEg,108,97,87
1,--3JQ4MlO-jHT9xbo7liug,165,134,122
2,--3P3fjPrScy06XVLi4vhg,119,101,82
3,--4DfhW2sJvjeC90KEDX3g,139,126,111
4,--5V3mRlhb3zqE5yF4Gpaw,108,98,88


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for filename in glob.glob("/content/drive/MyDrive/test/20GZaQkAup3g1U9ENd-bcQ.jpg"):
    image = Image.open(filename) 
    img_name = os.path.basename(filename)
    ## Number of prime color we gonna extrace
    num_colors=1

    #Generate small image for calculation
    small_image=image.resize((20,20))

    result=small_image.convert('P',colors=num_colors)
    result=result.convert('RGB')

    main_colors = result.getcolors(20*20)
    main_rgb = main_colors[0][1]
    print(main_rgb)


(255, 51, 51)


In [ ]:
#Open a file to write the pixel data
from itertools import product
from PIL import Image
import glob
import os
import csv

#Open a file to write the pixel data
with open('output_file.csv', 'w', newline='') as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerow(["img_name", "R", "G", "B"])

    #Path to file 
    for filename in glob.glob("*.jpg"):
        im = Image.open(filename) 
        img_name = os.path.basename(filename)

        #Load the pixel info
        pix = im.load()

        #Get a tuple of the x and y dimensions of the image
        width, height = im.size

        print(f'{filename}, Width {width}, Height {height}') # show 

        #Read the details of each pixel and write them to the file
        csv_output.writerows([img_name, *pix[x,y]] for x, y in product(range(width), range(height)))

--3JQ4MlO-jHT9xbo7liug.jpg, Width 400, Height 400


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
